In [1]:



import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np

# Get a list of S&P 500 tickers
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = sp500['Symbol'].tolist()

# Set the start and end dates for the historical data
start_date = dt.datetime.now() - dt.timedelta(days=3650*2) # last ten years
end_date = dt.datetime.now()

# Define a function to get the historical monthly data and simulate ESG scores for a given ticker
def get_data_and_esg(ticker):
    # Download the historical data from Yahoo Finance
    data = yf.download(ticker, start=start_date, end=end_date, interval='1mo')
    # Remove any rows with missing values
    data.dropna(inplace=True)
    # Define the ESG score array
    esg = np.random.randint(0, 101, size=len(data)//6) # simulate bi-annual updates
    esg = np.repeat(esg, 6) # repeat the same score for each month in the bi-annual period
    esg = np.append(esg, np.repeat(esg[-1], len(data) % 6)) # repeat the last score for any remaining months
    # Return a dataframe with the 'Close' column and the simulated ESG scores
    return pd.DataFrame({'Close': data['Close'], 'ESG': esg}, index=data.index)

# Loop through all tickers and download the data and simulate ESG scores
all_data = pd.DataFrame()
for ticker in tickers:
    print('Downloading data and simulating ESG for', ticker)
    try:
        data = get_data_and_esg(ticker)
        data.columns = pd.MultiIndex.from_product([[ticker], data.columns])
        all_data = pd.concat([all_data, data], axis=1)
    except:
        pass

# Save the data to a CSV file
all_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ABBV: Exception('Lost data during merge despite all attempts to align data (see above)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 co

In [2]:
#We create a rolling variance, mean and covariance, to measue stock volatility, and update our portfolio every 6 months, based on a weighted esg score of 80

#size of portfolio 100 stocks

#Weighted average combination of esg score = 80

# Other solution we cut off companies with esg score below 80

In [8]:
import pandas as pd
import numpy as np
import cvxpy as cp

# Load the data
data = all_data
start_date = dt.datetime.now() - dt.timedelta(days=3650*2) # last ten years
end_date = dt.datetime.now()

# Define the input parameters

esg_threshold = 79
num_months = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days // 30
print(data.loc[end_date])
selected_data = data.loc[start_date:end_date, data.loc[end_date].astype(int) > esg_threshold].astype(float)
returns = selected_data.pct_change().dropna()

# Define the objective function to be maximized, which is the negative Sharpe ratio
n = len(selected_data.columns)
w = cp.Variable(n)
ret = returns.mean() @ w
risk = cp.quad_form(w, returns.cov())
lambda_ = cp.Parameter(nonneg=True)
problem = cp.Problem(cp.Maximize(ret - lambda_ * risk), [cp.sum(w) == 1, w >= 0])

# Compute the efficient frontier for various values of lambda
MIN_LAMBDA = 0
MAX_LAMBDA = 5
NUM_POINTS = 100
lambdas = np.linspace(MIN_LAMBDA, MAX_LAMBDA, NUM_POINTS)
returns_list = []
risk_list = []
for l in lambdas:
    lambda_.value = l
    problem.solve()
    returns_list.append(ret.value)
    risk_list.append(cp.sqrt(risk).value)

# Compute the optimal portfolio for the given value of lambda
lambda_.value = 1
problem.solve()
optimal_weights = w.value

# Print the results
print("Optimal weights:", optimal_weights)
print("Expected return:", ret.value)
print("Volatility:", cp.sqrt(risk).value)


ValueError: Index Date invalid